In [ ]:
from fastai.vision.all import *
from fastai.metrics import *

In [ ]:
sys.path.insert(0,'../input/dm-nfnet')
import timm


class NFNet(nn.Module):
    def __init__(self, num_classes=11, model_nr=3, pretrained=True):
        super(NFNet, self).__init__()
        self.model = timm.create_model(f'dm_nfnet_f{model_nr}', pretrained=pretrained)
        self.model.head.fc = nn.Linear(self.model.head.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
dataset_path = Path('../input/ranzcr-clip-catheter-line-classification')
train_df = pd.read_csv(dataset_path/'train.csv')
train_df['path'] = train_df['StudyInstanceUID'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.drop(columns=['StudyInstanceUID'])

Tune model_nr and data augmentation. Batch size and image size are automatically adapted to the capacity of the Kaggle GPU

In [ ]:
img_sizes = [192, 224, 256, 320, 384, 416, 448]
model_nr = 0 # Between 0 and 5
partial_data = .05
size = img_sizes[model_nr]

For test runs use partial_data by uncommenting the commented line and commenting the line above

In [ ]:
aug_intensity = 1.1
item_tfms = RandomResizedCrop(size * 2, min_scale=.85, ratio=(.75, 1.33333))
batch_tfms = [*aug_transforms(mult=aug_intensity, do_flip=True, flip_vert=True, max_rotate=45, size=size, max_warp=0), Normalize.from_stats(*imagenet_stats)]

db = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(encoded=True, vocab=list(train_df.columns[:11]))),
               #splitter = RandomSplitter(.01),
               splitter=RandomSubsetSplitter(partial_data * .85, partial_data * .15),
               get_x = ColReader(12),
               get_y = ColReader(list(range(11))),
               item_tfms = item_tfms,
               batch_tfms = batch_tfms)

In [ ]:
bs = [128, 72, 36, 14, 6, 4]
dls = db.dataloaders(train_df, batch_size=bs[model_nr])
m = NFNet(model_nr=model_nr)
l = Learner(dls, m, loss_func=nn.BCEWithLogitsLoss(), metrics=[accuracy_multi]).to_fp16()

In [ ]:
l.fine_tune(4, base_lr=2e-5)

In [ ]:
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['PatientID'] = 'None'
sample_df['path'] = sample_df['StudyInstanceUID'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
sample_df = sample_df.drop(columns=['StudyInstanceUID'])
test_dl = dls.test_dl(sample_df, batch_size = int(bs[model_nr] / 2))

In [ ]:
l = l.to_fp32()
preds, _ = l.tta(dl=test_dl, n=2, beta=0.25)

In [ ]:
submission_df = sample_df.copy()
label_names = list(train_df.columns[:11])
for i in range(len(submission_df)):
    for j in range(len(label_names)):
        submission_df.iloc[i, j+1] = preds[i][j].numpy().astype(np.float32)

submission_df.to_csv(f'submission.csv', index=False)
print(submission_df.head(10))